## Demo notebook to check out the Google Places API and the results

In [1]:
import requests
from openai import OpenAI
import os

### Define functions

In [24]:
#get nearby restaurants
def get_nearby_restaurants(api_key, location, radius=200, keyword='restaurant', num_results=5):
    '''
    Returns the restaurants from the API call based on location, radius
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    results = response.json().get('results', [])

    # Limit the number of results to the top 'num_results'
    results = results[:num_results]
    
    return results


def get_place_details(api_key, place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})

    return result


def generate_summary(client, reviews):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"Summarize the reviews for this place:\n{review_text}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary
    

### Set api keys from env

In [3]:
#get api keys from env file
google_api_key = os.environ.get("GOOGLE_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
if google_api_key is not None:
    print("Google key found!")
if openai_api_key is not None:
    print("OpenAI key found!")

Google key found!
OpenAI key found!


### Search for places using Google Places API

In [21]:
# Munich center coordinates (for example)
munich_center = '48.1363964,11.5609501'

# Set radius and keyword
restaurants = get_nearby_restaurants(google_api_key, munich_center, radius=200, keyword='restaurant', num_results=3)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))

Restaurant name:  La Vecchia Masseria
Rating:   4
Restaurant name:  Schiller Bräu
Rating:   4.6
Restaurant name:  Münchner Stubn
Rating:   4.3


### Generate Summary with GPT3.5

In [22]:
from openai import OpenAI


#Client, automatically searches for OPENAI_API_KEY in environment and sets as key
client = OpenAI()


#Iterate over restaurants: We want to generate a review for each one
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    print("Number of available reviews: ", len(reviews))
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary(client, reviews)
    print(f"Summary: {summary}\n")

Number of available reviews:  5
Restaurant name:  La Vecchia Masseria
Summary: Overall, the reviews for this place are positive. The restaurant is described as lovely and quaint, with a bustling atmosphere due to families. It is considered a hidden gem located on a quiet street near the Aldstadt. The food is praised as soulful and amazing, with classic dishes like Bolognese and Gorgonzola being perfect. Prices are fair and typical for a restaurant of this type, although they have recently increased. It is recommended to make a reservation, especially if going later in the day. The ambience is nice, although it can be loud when full. The Christmas decorations were enjoyed by some reviewers. The outside seating is pleasant, and the restaurant has a good vibe and service. The food quality is generally good. One reviewer considers it their favorite pizza restaurant in Munich for the past six years. Al fresco dining in mid-June was loved by another reviewer, who found the food quite good an

## Prompt trials

In [26]:
def generate_summary_prompt(client, prompt):

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary

In [14]:
#Concatenate all reviews


"""
prompt1 = f"Summarize the reviews for this place as short and easy to understand as possible:\n{review_text}"
prompt2 = f"Summarize the reviews for this restaurant please. For this, please provide positive and negative aspects with bulletpoints. Positive ones should start with 'Pros:' and negative ones with 'Cons: ', These are the reviews:\n{review_text}"
prompt3 = f"Can you give me the most common positive and negative aspects in the reviews of this restaurant:\n{review_text}"
# prompt4 = f"Summarize the reviews for this place:\n{review_text}"
"""

print("PROMPT 1: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    print("review_text")
    prompt1 = f"Look through these reviews and summarize all of them in 2 to 3 sentences with the information and experiences that might be the most interesting to potential customers:\n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt1)
    print(f"Summary: {summary}\n")





PROMPT 1: 
review_text
Restaurant name:  La Vecchia Masseria
Summary: This restaurant is described as a hidden gem with a lovely atmosphere and amazing soulful food. Prices are reasonable and the outside seating area is praised. Some customers mention enjoying the Christmas decorations while others mention the consistent quality of the food over the years. The service is described as efficient.

review_text
Restaurant name:  Münchner Stubn
Summary: This restaurant is busy but has responsive and professional servers. They have a variety of local specialties and a great variety of beers on tap. The food is decent, well-presented, and comes in generous portions. They have a German and English menu and the staff is friendly and helpful. The vibe is fun with live music during Oktoberfest. The only negative is that one waiter was rude and rushed, but the overall dining experience is great. It's a cozy, moderately priced restaurant with efficient service and delicious food. It's conveniently 

In [16]:
print("PROMPT 2: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    prompt2 = f"Look through these reviews and provide a summary in form of bulletpoints about the most common opinions:\n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt2)
    print(f"Summary: {summary}\n")


PROMPT 2: 
Restaurant name:  La Vecchia Masseria
Summary: - The restaurant has a lovely and quaint atmosphere, with a bustling atmosphere due to families.
- The location is considered a hidden gem, close to the Aldstadt but a bit of a walk away.
- The food is soulful and amazing, with classics like Bolognese and Gorgonzola being perfect choices.
- Prices have increased recently but are still typical for a similar restaurant.
- It is recommended to make a reservation, although going early may allow one to skip it.
- All pizzas are €8.99 on Sundays.
- The ambience is nice, but it can be a bit loud when the restaurant is full.
- The Christmas decorations add to the enjoyable atmosphere.
- The prices are fair and not considered a rip-off for tourists.
- The overall quality of food is great, with good service.
- The outside seating in a quiet street is very nice, with a good vibe and service.
- Basic dishes have good quality, and the reviewer would return for beer and the atmosphere.
- The 

In [19]:
print("PROMPT 3: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    prompt = f"Look through these reviews and provide a 3 sentence summary, highlighting the following topics: food, ambience, service, price, consistency, creativeness, size. Please also provide one meal/option that seems the most popular, if there is one. These are the reviews:  \n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt)
    print(f"Summary: {summary}\n")

PROMPT 3: 
Restaurant name:  La Vecchia Masseria
Summary: Summary:

- Food: The food is described as soulful, amazing, and good quality. Pasta dishes like Bolognese and Gorgonzola are particularly praised.
- Ambience: The ambience is nice, with a great atmosphere and Christmas decorations. Outside seating on a quiet street is also enjoyed.
- Service: The service is efficient and responsive, but not overly warm or attentive.
- Price: Prices are fair and typical for a similar restaurant, with recent increases noted.
- Consistency: One reviewer mentions that the restaurant has maintained the same quality and workers for several years.
- Creativeness: No specific comments about the creativity of the food.
- Size: No specific comments about the portion sizes.

Most popular meal/option: All pizzas are €8.99 on Sundays.

Restaurant name:  Münchner Stubn
Summary: Summary:
- The food at this German restaurant is decent in flavor with generous portions and great presentation. They offer a variet

## Notes on experimentation

- text or bullet points?
- how long should a summary be, e.g. 3-4 sentences, 6-10 bullet points?
- repetitive sentences
- highlighting of specific criteria, e.g. food, ambience, price, service
- popular meal option?
- how to deal with translated reviews? Use translation or omit?

## Detailed Look into Reviews from Places API

In [28]:
#Use Place ID of known place:
place_id = 'ChIJe-rD4fV1nkcRNk5AlWoeV5k'


place_details = get_place_details(google_api_key, place_id)

print(place_details)
for key in place_details.keys():
    print(key)
# print(place_details.keys())

{'address_components': [{'long_name': '4', 'short_name': '4', 'types': ['street_number']}, {'long_name': 'Josephspitalstraße', 'short_name': 'Josephspitalstraße', 'types': ['route']}, {'long_name': 'Altstadt-Lehel', 'short_name': 'Altstadt-Lehel', 'types': ['sublocality_level_1', 'sublocality', 'political']}, {'long_name': 'München', 'short_name': 'München', 'types': ['locality', 'political']}, {'long_name': 'Kreisfreie Stadt München', 'short_name': 'Kreisfreie Stadt München', 'types': ['administrative_area_level_3', 'political']}, {'long_name': 'Oberbayern', 'short_name': 'Oberbayern', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Bayern', 'short_name': 'BY', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Germany', 'short_name': 'DE', 'types': ['country', 'political']}, {'long_name': '80331', 'short_name': '80331', 'types': ['postal_code']}], 'adr_address': '<span class="street-address">Josephspitalstraße 4</span>, <span class="postal-c

In [29]:
place_details['reviews']

[{'author_name': 'Rashmita Baruah',
  'author_url': 'https://www.google.com/maps/contrib/113123533601990585744/reviews',
  'language': 'en',
  'original_language': 'en',
  'profile_photo_url': 'https://lh3.googleusercontent.com/a-/ALV-UjU6m5fcYfQ_-dD5FB0sdKKHk2pOnL7yvW_K539JAU4FnUo9=s128-c0x00000000-cc-rp-mo',
  'rating': 3,
  'relative_time_description': '2 months ago',
  'text': 'I liked their concept of mini burgers, as you get to try different varieties of mini sizes. However, the taste of the burgers are not that amazing. Some of the burgers are too saucy which ruined the overall taste. Service was good. The only downside is that they the tables can be reserved only for 1.5 hours which according to me is too short, especially when you’re catching up with friends after a long time.',
  'time': 1696690147,
  'translated': False},
 {'author_name': 'Alberto Serrano Monferrer',
  'author_url': 'https://www.google.com/maps/contrib/103995167760940536809/reviews',
  'language': 'en',
  'o

In [30]:
reviews = place_details.get('reviews', [])
review_text = "\n".join(review.get('text', '') for review in reviews)
prompt = f"Look through these reviews and provide a summary in 3 to 4 sentences. Cover the following criteria: Food, service, ambience, creativity, price.\n{review_text}"
print(review_text)

#Generate the summary
summary = generate_summary_prompt(client, prompt)
print(f"Summary: {summary}\n")

I liked their concept of mini burgers, as you get to try different varieties of mini sizes. However, the taste of the burgers are not that amazing. Some of the burgers are too saucy which ruined the overall taste. Service was good. The only downside is that they the tables can be reserved only for 1.5 hours which according to me is too short, especially when you’re catching up with friends after a long time.
This is an interesting and informal restaurant, specialized in all types of hamburguers of small size. It's a nice option for having lunch if you are visiting the city center of München.
Furthermore, there are options of menus which include two or three hamburguers, a little drink and a complement like green salad or chips. We had 2 hamburguers menu and also a 3 hamburguers one, which was enough for lunch.
Mara, who was our waiter, was extremely warm and explained everything to choose our food.
By the way, it is important to make a reservation before!
A wonderful restaurant in the 

## Useful Information Keys in place details:

### Standard information
- formatted_address: Address ready to display as information
- business status: is place operational or not
- current_opening_hours
- delivery/takeout and/or dine_in
- reservable: Can you reserve tables?
- rating

### Additional information:
- serves_breakfast
- serves_brunch
- serves_lunch
- serves_vegetarian_food
- serves_beer
- serves_wine
- wheelchair_accessible_entrance